In [ ]:
import pandas as pd
import glob
import json
import matplotlib.pyplot as plt
import os
import re
import seaborn as sns

sns.set_style("white", {'axes.grid' : True})

In [ ]:
resultdir = "/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative"
trainre = re.compile(r'^.+\/iterative\/(?P<modelname>[^_]+)_(?P<imgsz>[\d]+)imgsz_(?P<camera>[^_]+)_(?P<epochs>[\d]+)epochs$')

model_parent = "yolov"
train_df = pd.DataFrame()

for trainresult in glob.glob(f"{resultdir}/*epochs"):
    match = trainre.match(trainresult)
    if match is None:
        print(F"didn't match directory name {trainresult}")
        continue
    params = match.groupdict()
    new_result = params.copy()
    new_result['modelname'] = f"{model_parent}{new_result['modelname']}"

    if not os.path.exists(f"{trainresult}/results.csv"):
        print(f"{trainresult}/results.csv doesn't exist right now, skipping")
        continue
    
    df = pd.read_csv(f"{trainresult}/results.csv")
    df['imgsz'] = new_result['imgsz']
    df['max_epochs'] = new_result['epochs']
    df['camera'] = new_result['camera']
    df['modelname'] = new_result['modelname']
    df = df.astype({'epoch':'int', 
                    'time': 'float', 
                    'train/box_loss': 'float', 
                    'train/cls_loss': 'float',
                    'train/dfl_loss': 'float',
                    'metrics/recall(B)': 'float',
                    'metrics/mAP50-95(B)': 'float',
                    'val/box_loss': 'float',
                    'val/cls_loss': 'float',
                    'val/dfl_loss': 'float',
                    'lr/pg0': 'float',
                    'lr/pg1': 'float',
                    'lr/pg2': 'float',
                    'imgsz': int,
                    'max_epochs': int,
                   })
    df.dropna(subset=['train/box_loss', 'train/cls_loss', 'train/dfl_loss'], inplace=True)
    train_df = pd.concat([train_df, df])

In [ ]:
#pd.set_option('display.max_rows', None)
pd.reset_option('^display.', silent=True)

train_df.rename(columns={'imgsz': "Image Size (px)", 'epoch': 'Epoch',
#                       'precision': 'Precision', 'recall': 'Recall',
                         'camera': 'Camera', 'modelname': 'Model Name',
                         'train/box_loss': 'Box', 
                         'train/cls_loss': 'Classification',
                         'train/dfl_loss': 'Distribution Focal'},
                         inplace=True)

# Define the custom sort order
model_order = ['yolov11n', 'yolov11s', 'yolov11m']

train_df['Model Name'] = pd.Categorical(train_df['Model Name'], categories=model_order, ordered=True)

train_df.sort_values(['Camera', 'Model Name', 'Image Size (px)', 'Epoch'], inplace=True)

# calculate epoch times
train_df['epoch_duration'] = train_df['time'].diff()

# reset difference of first epoch to zero
train_df.loc[train_df['Epoch'] == 1, 'epoch_duration'] = train_df.loc[train_df['Epoch'] == 1, 'time']
# remove intermediate one with negative times because of restart
train_df.loc[train_df['epoch_duration'] < 0, 'epoch_duration'] = train_df.loc[train_df['epoch_duration'] < 0, 'time']

#train_df['epoch_duration_f'] = train_df['epoch_duration'] if (train_df['Epoch'] > 1) else 0
#train_df.where(train_df['Epoch'] > 1,  train_df['epoch_duration'], axis=1, inplace=True)
print(train_df.dtypes)
display(train_df)

mask_list = []
mask_list.append(train_df['Camera'] == "IP8M-H-SW") 
mask_list.append(train_df['Image Size (px)'] == "736") 
#for s in [ 3840, ]:
#    mask_list.append(train_df['Image Size (px)'] == s) 
#    #display(train_df[ join_mask(mask_list) ])
#    #print(f"{s}: {len( train_df[ join_mask(mask_list) ] )}" )
#    mask_list.pop()



plot_metrics = ['Box', 'Classification','Distribution Focal' ]

def join_mask( mask_list ):
    full_mask = None
    for m in mask_list:
        if full_mask is None:
            full_mask = m
        else:
            full_mask = full_mask & m
    return full_mask

display( train_df[join_mask(mask_list)] )

# Plot training Loss

In [ ]:
# iterate over cameras
mask_list = []
for camera in train_df['Camera'].unique().tolist():
    mask_list.append(train_df['Camera'] == camera) 
    
    # iterate over model
    for model in train_df[join_mask(mask_list)]['Model Name'].unique().tolist(): 
        mask_list.append((train_df['Model Name'] == model))

        current_df = train_df[ join_mask( mask_list ) ]
        
        #display(current_df)
        for metric in plot_metrics:
            #print(f"{camera}, {model}, {metric}")
            #fig = plt.figure()
            #axs = sns.lineplot(data=current_df, x='Epoch', y=metric, hue='Image Size (px)', palette='colorblind' )
            #axs.set_title(f"Training Loss {metric} for {model} (camera {camera})") 
            #if metric == "Classification":
            #    axs.set_ylim([0,10])
            #plt.show()
            #plt.close()
            pass
        mask_list.pop()
    mask_list.pop()

In [ ]:
for model in ["yolov11n"]: #, "yolov11s", "yolov11m"]:
    for imgsize in [736, 3840]: #[736, 1440, 2880, 3840 ]:
        mask_list = []
        mask_list.append( train_df['Model Name'] == model )
        mask_list.append( train_df['Image Size (px)'] == imgsize )
        
        cameras = train_df[ join_mask(mask_list) ]['Camera'].unique().tolist()
        if len(cameras) == 1:
            continue
            
        fig, axs = plt.matplotlib.pyplot.subplots(nrows=len(cameras), ncols=1, sharex=True, sharey=False, figsize=(6,8))
        for i, camera in enumerate(cameras):
            camera_name = camera if camera[0].isupper() else camera.capitalize()
            mask_list.append((train_df['Camera'] == camera))
            current_df = train_df[ join_mask( mask_list ) ]
        
            melt_df = pd.melt(current_df, id_vars=['Epoch'], value_vars=plot_metrics, var_name='Loss', value_name='Loss Value')
            #display(melt_df)
            axs[i] = sns.lineplot(data=melt_df, x='Epoch', y='Loss Value', palette='colorblind', hue='Loss', ax=axs[i])
            axs[i].set_title(f"{camera_name} Dataset") 
        
            mask_list.pop()
            axs[i].set_ylim([0,5])
        
        fig.suptitle(f"Training Loss for {model} using {imgsize} pixel Images")
        fig.tight_layout()
        plt.savefig(os.path.join("/data/mtmoore/school/CSiML_AI395T/final_project/plots", f"yolov11_training_losses_{imgsize}px_{model}.png"))


# Plot Epoch Time

In [ ]:
epoch_duration_mean_df = train_df.groupby(by=["Camera", "Model Name", "Image Size (px)"], observed=False).agg( Mean_Duration=('epoch_duration', 'mean')  ).reset_index()

epoch_duration_mean_df.rename(columns={'Mean_Duration': "Mean Duration"}, inplace=True)

#cameras = epoch_duration_mean_df['Camera'].unique().tolist()
cameras  = ["IP8M-H-NW"]
fig, axs = plt.matplotlib.pyplot.subplots(nrows=len(cameras), ncols=1, sharex=True, sharey=True)
#for i, camera in enumerate(cameras):
for i, camera in enumerate(cameras):
    curr_ax = axs
    if isinstance(axs, list):
        curr_ax = axs[i]
    camera_name = camera if camera[0].isupper() else camera.capitalize()
    
    current_df = epoch_duration_mean_df[ epoch_duration_mean_df['Camera'] == camera ]
    display(current_df)
    curr_ax = sns.barplot(data=current_df, x="Model Name", y="Mean Duration", hue='Image Size (px)', ax=curr_ax, palette='colorblind',)
    curr_ax.set_title(f"Dataset {camera_name}")
    curr_ax.set_ylabel("Avg. Epoch Duration (sec)")
fig.suptitle("YOLOv11 Training Time Per Epoch", fontsize=13)
fig.tight_layout()

plt.savefig(os.path.join("/data/mtmoore/school/CSiML_AI395T/final_project/plots", f"yolov11_epoch_training_time.png"))

#for camera in train_df['Camera'].unique().tolist():
#    mask_list.append(train_df['Camera'] == camera) 
    
#    # iterate over model
#    for model in train_df[join_mask(mask_list)]['Model Name'].unique().tolist(): 
#        mask_list.append( train_df['Model Name'] == model )
        
#        # iterate over model
#        for imgsize in train_df[join_mask(mask_list)]['Image Size (px)'].unique().tolist(): 
#            mask_list.append( train_df['Image Size (px)'] == imgsize )